# 사전학습 된 모델 불러오기

In [ ]:
from ultralytics import YOLO
import torch

model = YOLO("models/yolo11n.pt")  # COCOdataset으로 학습된 데이터셋

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# 데이터셋 yaml 경로
DATA_YAML_PATH = "C:/project_sep/modeling/YOLO/yolo11n/drug-detection-1/data.yaml"  # 내가 fork 해온 drug 데이터셋
# COCODATASET 으로 사전 학습된 yolo11n 모델을 내 데이터셋에 맞춰서 파인튜닝을 진행

# 학습
results = model.train(
    data=DATA_YAML_PATH,
    epochs=50,  # baseline: 50 epoch
    batch=8,
    imgsz=640,
    device=0,  # 0: 첫 번째 GPU, 없으면 CPU
)

Ultralytics 8.3.199  Python-3.10.18 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/project_sep/modeling/YOLO/yolo11n/drug-detection-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=models/yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overl

In [ ]:
metrics = model.val()  #
print(metrics)  # Precision, Recall, mAP 출력

Ultralytics 8.3.199  Python-3.10.18 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 3142.7419.7 MB/s, size: 655.1 KB)
val: Scanning C:\project_sep\modeling\YOLO\yolo11n\drug-detection-1\valid\labels.cache... 21 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 21/21  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 1.1it/s 2.7s0.7ss
                   all         21         61      0.983          1      0.995      0.864
Speed: 2.1ms preprocess, 23.5ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to C:\YOLOproject\runs\detect\train42
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000014EBB

# 내 이미지로 테스트 해보기

In [ ]:
input_dir = "C:/project_sep/modeling/YOLO/yolo11n/images/original"  # 폴더에 이미지 넣어야함(약 데이터)
output_dir = "C:/project_sep/modeling/YOLO/yolo11n/images/results"  # 결과값 출력

import os

os.makedirs(output_dir, exist_ok=True)

results = model.predict(
    source=input_dir,
    conf=0.5,  # 우리는 0.8 이상만 된 걸 줘야하는데, 일단은 0.5
    save=True,
    project=output_dir,
    name="",  # 하위폴더 생성 방지
)

print(f"결과 이미지가 {output_dir} 폴더에 저장되었습니다.")


image 1/5 C:\project_sep\modeling\YOLO\yolo11n\images\original\IMG_5239_jpg.rf.58386d5f40603c740d2c2c35842d8f3b.jpg: 640x640 5 drugs, 37.6ms
image 2/5 C:\project_sep\modeling\YOLO\yolo11n\images\original\IMG_5247_jpg.rf.dce09eb5fc447e28c7386c342e13ff16.jpg: 640x640 1 drug, 33.4ms
image 3/5 C:\project_sep\modeling\YOLO\yolo11n\images\original\mypill_resized.jpg: 640x640 4 drugs, 35.6ms
image 4/5 C:\project_sep\modeling\YOLO\yolo11n\images\original\test1.jpg: 640x384 4 drugs, 45.1ms
image 5/5 C:\project_sep\modeling\YOLO\yolo11n\images\original\test2.jpg: 640x384 4 drugs, 35.6ms
Speed: 3.9ms preprocess, 37.5ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 384)
Results saved to C:\project_sep\modeling\YOLO\yolo11n\images\results\predict
결과 이미지가 C:/project_sep/modeling/YOLO/yolo11n/images/results 폴더에 저장되었습니다.


# 순서 정리

## 1. `yolo11n.pt` 불러오기 (COCO pretrained)
## 2. `data.yaml` 연결 (내 데이터셋)
## 3. `model.train(...)` 실행 -> fine tuning
## 4. `model.val()` 선능 확인
## 5. `model.predict()` -> 내 사진/폴더 테스트